# 4 - Testing Recall vs Noise

This notebook creates a bulk of the simulated data for the Toy model section of the paper.

Here we test the recall of diffraction vectors vs noise.

Carter Francis | csfrancis@wisc.edu | 2024-01-15

In [1]:
import numpy as np
from skimage.draw import disk
import matplotlib.pyplot as plt
from dask_image.ndfilters import gaussian_filter
from skimage.morphology import disk as d2

from orix.crystal_map import Phase
from orix.quaternion import Rotation
import hyperspy.api as hs

from utils import *

# Creating Data

In [ ]:
hs.set_log_level("ERROR")
hs.preferences.General.show_progress_bar=False
num_electrons =[0.001, 0.002,0.004,0.005,.01] 
for i in range(6):
    sim = CrystalSTEMSimulation(Al_Structure, real_space_pixels=256, num_crystals=40)
    with open("AllResults/num_electrons"+str(i)+".txt","w+") as f:
        f.write("num_crystals, recall, false_positives, electrons_per_vector \n")
        from skimage.draw import disk
        mask = np.ones((64,64), dtype=bool)
        rr,cc = disk((32,32), 10)
        mask[rr,cc]=False
        for s, n in zip(simulations,num_electrons):
            arr = s.make_4d_nano(num_electrons=n)
            ground_truth_vectors = s.vectors
            print(np.sum(arr[:,:, mask])/(256 * 256), "Average Electrons per Frame")
            signal = hs.signals.Signal2D(arr)
            signal.set_signal_type("electron_diffraction")
            signal = signal.as_lazy()
            signal.rechunk((32,32))

            filtered = signal.filter(gaussian_filter, sigma=(1,1,0,0))
            template = filtered.template_match_disk(disk_r=5, subtract_min=False,show_progressbar=False)
            template.data[:,:,:, 0:5]=0
            template.data[:,:,:,-5:]=0
            template.data[:,:, 0:5, :]=0
            template.data[:,:,-5:, :]=0
            pks = template.find_peaks(threshold_abs=0.4, interactive=False,show_progressbar=False )
            from pyxem.signals.diffraction_vectors import DiffractionVectors
            vect = DiffractionVectors.from_peaks(pks, center= (32, 32),
                                                     calibration=(1/32,1/32))
            vect.compute()
            filt = vect.filter_magnitude(.05,
                                         .8,
                                             show_progressbar=False )
            ground_truth_vectors = ground_truth_vectors[(ground_truth_vectors[:,2]**2+ground_truth_vectors[:,3]**2)**0.5>0.05]
            ground_truth_vectors = ground_truth_vectors[(ground_truth_vectors[:,2]**2+ground_truth_vectors[:,3]**2)**0.5<0.8]

            from scipy.spatial import KDTree
            flat = filt.flatten_diffraction_vectors()
            ground_truth_vectors[:,2:4] = ground_truth_vectors[:,2:4]*32
            flat.data[:,2:] = flat.data[:,2:]*32
            gt_tree = KDTree(ground_truth_vectors[:,0:4])
            flat_tree = KDTree(flat.data)

            false_positives = [len(i) == 0 for i in gt_tree.query_ball_point(flat.data, r=9)]

            is_bigger = [len(i)>1 for i in flat_tree.query_ball_point(ground_truth_vectors[:,:4], r=2)]
            r = np.sum(is_bigger)/len(is_bigger)
            fp = np.sum(false_positives)/ len(false_positives)
            electrons_per_vector = np.mean(ground_truth_vectors[:,-1])*n*69
            print("electrons_per_vector", electrons_per_vector)
            
            print("Recall ", r) 
            print("False Positives", fp)
            f.write(str(n)+", " + str(r)+", "+ str(fp)+", "+ str(electrons_per_vector) +"\n")

## Plotting

In [ ]:
recall = [np.loadtxt("AllResults/num_electrons"+str(i)+".txt", delimiter=",", skiprows=1) for i in range(6)]
mean_recall = np.mean(recall, axis=0)
std_recall = np.std(recall, axis=0)

In [ ]:
fig = plt.figure(figsize=(5,5))

axs = fig.add_axes((.15,.15, 0.8,0.7))
axs.errorbar(mean_recall[:,-1],
             mean_recall[:,1]*100,
             std_recall[:,1]*100, 
             capsize=2, color="black",ls="none",
             marker="o", markersize=5
            )
axs.set_xlabel("Average Electrons per Vector")
axs.set_ylabel("Recall Percentage")
axs.set_ylim(0,104)

axs2 = fig.add_axes((.2,.2, 0.2,0.2))

axs3 = fig.add_axes((.45,.2, 0.2,0.2))

axs4 = fig.add_axes((.7,.2, 0.2,0.2))

for ax in [axs2,axs3,axs4]:
    ax.set_xticks([])
    ax.set_yticks([])

simulations[0].plot_example_dp(ax=axs2,threshold=0.6, num_electrons=0.001)
simulations[4].plot_example_dp(ax=axs3,threshold=0.6, num_electrons=0.004)
simulations[-1].plot_example_dp(ax=axs4, threshold=0.6, num_electrons=0.01)



for a, i,c in zip([axs2, axs3, axs4], [0,2,-1], corners):
    axs.annotate("", xy=(mean_recall[:,-1][i],-0.5), xytext=(c, .16), xycoords="data",
                 textcoords="figure fraction",
                 arrowprops=dict(facecolor='black',arrowstyle="-", lw=1.5),annotation_clip=False)
    axs.annotate("", xy=(mean_recall[:,-1][i],-0.5), xytext=(c+0.18, .16), xycoords="data",
                 textcoords="figure fraction",
                 arrowprops=dict(facecolor='black',arrowstyle="-", lw=1.5),annotation_clip=False)
fig.savefig("Figures/Figure3-RecallVsNoise.png", bbox_inches="tight" )
plt.show()